In [1]:
import numpy as np
import sys, os  
import quippy as qp

In [2]:
from ase.io import read,write
from ase.visualize import view

In [38]:
sys.path.insert(0,'../')
sys.path.insert(0,'../tools')
from GlobalSimilarity import get_environmentalKernels, get_globalKernel
from CV import CrossValidation
from krr import KRR,dump_json,load_json,dump_data,load_data,score

# Compute a kernel and save it

In [26]:
frames = qp.AtomsList('./small_molecules.xyz',stop=100)

In [27]:
soap_params = dict(cutoff=3, nmax=6, lmax=6, gaussian_width=0.4,
                    cutoff_transition_width=0.5, centerweight=1.,nocenters=[],
                   chem_channels=True, is_fast_average=False,
                   islow_memory=False,nthreads=1,nchunks=1,nprocess=1)

environmentalKernels = get_environmentalKernels(atoms=frames,**soap_params)

HBox(children=(IntProgress(value=0, description=u'SOAP vectors'), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=5050), HTML(value=u'')))

In [30]:
kernel_params = dict(kernel_type='average', zeta=2, normalize_global_kernel=True)
globalKernel = get_globalKernel(environmentalKernels,**kernel_params)

In [31]:
prefix = './'
fn = 'my_kernel_matrix'
metadata = dict(soap_params=soap_params,fn=fn+'.npy',
                kernel_params=kernel_params)
dump_data(prefix+fn+'.json',metadata,globalKernel)

# Train and save a KRR model

In [43]:
params, Kmat = load_data('./my_kernel_matrix.json')

train = range(100)

Kmat_train = Kmat[np.ix_(train,train)]
y_train = np.load('./small_molecules-dHf_peratom.npy')[train]

In [44]:
model = KRR(sigma=1e-1,csi=1)
model.train(Kmat_train,y_train)

In [45]:
state = model.pack()
dump_json('./my_krr_model.json',state)

# Predict with saved model

In [46]:
params, Kmat = load_data('./my_kernel_matrix.json')

train = range(100)
test = range(50,100)

Kmat_test = Kmat[np.ix_(train,test)]

y_test = np.load('./small_molecules-dHf_peratom.npy')[test]

model_state = load_json('./my_krr_model.json')
model = KRR().unpack(model_state)

In [47]:
y_pred = model.predict(Kmat_test)

print 'MAE={:.3e} RMSE={:.3e} SUP={:.3e} R2={:.3e} CORR={:.3e}'.format(*score(y_pred,y_test))

MAE=1.380e-01 RMSE=2.061e-01 SUP=7.916e-01 R2=2.669e-01 CORR=6.396e-01


# Kfold CV

In [56]:
params, Kmat = load_data('./my_kernel_matrix.json',mmap_mode=None)
y = np.load('./small_molecules-dHf_peratom.npy')[:100]
params = dict(sigma=1e-1,csi=1)

In [57]:
scoreTest, err_scoreTest = CrossValidation(Kmat,y,params,Nfold=4,seed=10)
print 'MAE={:.3e} RMSE={:.3e} SUP={:.3e} R2={:.3e} CORR={:.3e}'.format(*scoreTest)

MAE=1.711e-01 RMSE=2.844e-01 SUP=1.002e+00 R2=1.710e-02 CORR=6.090e-01
